In [1]:
#NN Surrogate model class
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import gpytorch
import botorch 

# BO for Minimizing Emittance with 3 Variables (SQ, CQ, SOL)

In [2]:
#load injector model

### original tf/keras model ###
# Model = Surrogate_NN()

# Model.load_saved_model(model_path = '../models/', \
#                        model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
# Model.load_scaling()
# Model.take_log_out = False

### pytorch model ### 
Model = Surrogate_NN(pytorch=True)

Model.load_saved_model(model_path = '../models/', 
                       model_name = 'Surrogate_NN_PyTorch')

Model.load_scaling(scalerfilex = '../data/transformer_x_pytorch.pth', 
                   scalerfiley = '../data/transformer_y_pytorch.pth')
Model.take_log_out = False

## Objective Function

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient']
#input bounds
bounds = torch.tensor([[0.44, 0.55],[-0.02, 0.02], [-0.02, 0.02]])

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

In [4]:
def evaluate(config): 
    """
    D is input space dimensionality
    N is number of sample points
    :param config: input values of opt_var_names, torch.tensor, shape (N, D) 
    returns (1, N) 
    """
    N = config.shape[0]
    D = config.shape[1]
    
    # make input array of length model_in_list (inputs model takes)
    x_in = torch.empty((N,len(Model.model_in_list)))
    
    # fill in reference point around which to optimize
    x_in[:,:] = torch.as_tensor(ref_point[0])

    #set solenoid, CQ, SQ, matching quads to values from optimization step
    col = []
    for i in range(D):
        col.append(Model.loc_in[opt_var_names[i]]) #should make col a flat list of indices, e.g. [4, 6, 7]
    x_in[:, col] = config[:,:] 
    
    #output predictions
    x_in = x_in.float()
    y_out = Model.pred_machine_units(x_in)

    return -1*objective(y_out).detach() ## !! ASK ABOUT Trying to backward through the graph a second time, but the buffers have already been freed

def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return torch.sqrt(out1*out2)/1e-6 # in um units

## Gaussian Regression & Acquisition Function

In [5]:
def get_BO_point(x, f, bounds, beta=2.0, input_transform=None, outcome_transform=None, precision = None):
    """
    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    gp = botorch.models.SingleTaskGP(x, f, 
                                     outcome_transform=outcome_transform, 
                                     input_transform=input_transform)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,#5
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [6]:
#create initial samples within specified bounds
n_samples = 5
n_var = 3

train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.as_tensor(np.random.uniform(bounds[i,0],bounds[i,1],(n_samples,)))
print(train_x)

train_y = evaluate(train_x).reshape(-1,1)
print(train_y)

# transformer 
transformer_x = botorch.models.transforms.input.Normalize(n_var, bounds = bounds.transpose(0,1))
transformer_y = botorch.models.transforms.outcome.Standardize(1)

normed_bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)

tensor([[ 0.4922,  0.0072,  0.0199],
        [ 0.5402,  0.0063, -0.0175],
        [ 0.4577, -0.0119, -0.0158],
        [ 0.4809,  0.0194, -0.0150],
        [ 0.5441, -0.0109,  0.0085]])
tensor([[-2.0875],
        [-5.1006],
        [-1.8745],
        [-0.7227],
        [-4.6886]])


## Bayesian Optimization

In [7]:
n_steps = 30
best_y = torch.max(train_y)

for i in range(n_steps):  
    x_new, model = get_BO_point(train_x, train_y, 
                                bounds=bounds.transpose(0,1), 
                                input_transform=transformer_x, 
                                outcome_transform=transformer_y)

    train_x = torch.cat((train_x, x_new))
    new_y = evaluate(train_x[-1].reshape(1,-1)).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    if (new_y > best_y):
        best_y = new_y
        color = '\033[95m', '\033[0m'
    else: 
        color = '\u001b[30m', '\033[0m'
    
    print("iteration        target         SOL          SQ          CQ")
    print(f'{color[0]}{i+1}              {train_y[-1,0]:.5f}     {train_x[-1,0]:.5f}     {train_x[-1,1]:.5f}     {train_x[-1,2]:.5f}{color[1]}')

iteration        target         SOL          SQ          CQ
1              -1.06831     0.46008     0.01331     -0.00933
iteration        target         SOL          SQ          CQ
2              -1.11539     0.45950     0.02000     -0.02000
iteration        target         SOL          SQ          CQ
3              -0.78095     0.47614     0.02000     0.00001
iteration        target         SOL          SQ          CQ
4              -2.08438     0.45723     0.02000     0.01371
iteration        target         SOL          SQ          CQ
5              -0.59519     0.47793     0.00867     -0.00999
iteration        target         SOL          SQ          CQ
6              -0.78877     0.47298     0.01008     -0.02000
iteration        target         SOL          SQ          CQ
7              -0.69033     0.47351     0.01614     -0.01030
iteration        target         SOL          SQ          CQ
8              -0.76370     0.48455     0.01382     -0.00602
iteration        target         SO

In [8]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""

'\n#define acquisition function\nfrom botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement\nfrom botorch.optim import optimize_acqf\n\n#optimize\nn_steps = 45\nfor i in range(n_steps):\n    #best_normed_y = torch.max(normed_train_y)\n    UCB = UpperConfidenceBound(gp, beta=2.5)\n    #EI = ExpectedImprovement(gp, best_normed_y)\n\n    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)\n    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)\n\n    train_x = torch.cat((train_x, transformer_x.backward(candidate)))\n    normed_train_x = transformer_x.forward(train_x)\n\n    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)\n    train_y = torch.cat((train_y, new_y))\n    \n    print("iteration        target         varx         vary         varz")\n    print(f\'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-

In [9]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 216.97930908203125
199 148.77322387695312
299 102.99685668945312
399 72.24969482421875
499 51.58010482788086
599 37.67311096191406
699 28.30780029296875
799 21.995250701904297
899 17.736312866210938
999 14.860123634338379
1099 12.915842056274414
1199 11.600123405456543
1299 10.708847045898438
1399 10.10446548461914
1499 9.694156646728516
1599 9.41530990600586
1699 9.225595474243164
1799 9.096376419067383
1899 9.008256912231445
1999 8.948097229003906
Result: y = -0.008838691748678684 + 0.8491359949111938 x + 0.0015248217387124896 x^2 + -0.09224866330623627 x^3
